In [ ]:
import pandas as pd
import xlwt
import datetime as dt
from xlwt import Workbook
from googleapiclient.discovery import build

# Set API parameters.
api_key = "key"
channel_id = 'UCzEy7pi3B7TIS9cn_sdKK9A' # Gray Still Plays
youtube = build('youtube','v3', developerKey = api_key)

# Functions to call API to gather information.
class ytStats:
    def __init__(self, api_key, channel_id):
        self.api_key = api_key
        self.channel_id = channel_id

    # Gather general channel statistics.  
    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = channel_id
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title']  , 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'] ,
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount']  , 
                result_channel_views = response['items'][i]['statistics']['viewCount']  , 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    # Gather individual IDs to be used to gather information. 
    def get_video_ids(youtube, playlist_id):
        request = youtube.playlistItems().list(
            part = 'contentDetails' , 
            playlistId = playlist_id , 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails' , 
                    playlistId = playlist_id , 
                    maxResults = 50 , 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    # Gather details from each individual videos. 
    def get_video_details(youtube, video_ids):
        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics' , 
                id = ','.join(video_ids[i: i + 50])
            )
            response = request.execute()
            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title']  ,
                    result_video_id = video['id'] ,                
                    result_video_description = video['snippet']['description']  ,
                    result_video_upload_time = video['snippet']['publishedAt']  ,
                    result_video_views = video['statistics']['viewCount'] ,  
                    result_video_likes = video['statistics']['likeCount'] ,  
                    result_video_favorites = video['statistics']['favoriteCount'] ,  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)
        return all_data

# Call functions.
yt = ytStats
result_channel_statistics = yt.get_channel_statistics(youtube)#, channel_id)
playlist_id               = result_channel_statistics[0]['result_playlist_id']
video_ids                 = yt.get_video_ids(youtube, playlist_id)                # Time = 5.6s
video_details             = yt.get_video_details(youtube, video_ids)              # Time = 8.5s

In [ ]:
def create_data_frame(result_channel_statistics, video_details):
    # Create data frames.
    channel_data = pd.DataFrame(result_channel_statistics, columns = [
        'result_playlist_id' ,
        'result_channel_name' , 
        'result_channel_views' ,
        'result_channel_subscribers' ,
        'result_channel_total_videos' 
        ])
    initial_video_data = pd.DataFrame(video_details, columns = [
        'result_video_id' ,
        'result_video_upload_time' , 
        'result_video_name' , 
        'result_video_description' , 
        'result_video_views' , 
        'result_video_likes' , 
        'result_video_comments' 
        ])

    # Fromat data frames for initial_video_data.
    upload_time = initial_video_data.groupby('result_video_upload_time', as_index = False).size()
    published_dttm = pd.to_datetime(upload_time['result_video_upload_time']).dt    
    initial_video_data['published_date'] = upload_time['Dates'] = published_dttm.date
    initial_video_data['published_time'] = upload_time['Time']  = published_dttm.time
    del initial_video_data['result_video_upload_time']
    video_data = initial_video_data.reindex(columns = ['result_video_id', 'published_date', 'published_time', 'result_video_name','result_video_description','result_video_views','result_video_likes','result_video_comments'])
    video_data.columns = ['Video ID', 'Date', 'Time', 'Video Name', 'Video Description', 'Views', 'Likes', 'Comments', ]
    
    return channel_data, video_data

channel_data, video_data = create_data_frame(result_channel_statistics, video_details)

In [ ]:
# Individual IDs
channel_name = result_channel_statistics[0]['result_channel_name']
channel_subs = result_channel_statistics[0]['result_channel_subscribers']
channel_views = result_channel_statistics[0]['result_channel_views']
channel_vid_count = result_channel_statistics[0]['result_channel_total_videos']
todays_date = dt.datetime.now()

In [ ]:
wb = Workbook()

sheet1 = wb.add_sheet('Channel Stats') # sheet1.write(
sheet1.write(0, 0, 'name')
sheet1.write(0, 1, channel_name)

sheet1.write(1, 0, 'date')
sheet1.write(1, 1, 'subs')
sheet1.write(1, 2, 'views')
sheet1.write(1, 3, 'count')

sheet1.write(2, 0, todays_date)
sheet1.write(2, 1, channel_subs)
sheet1.write(2, 2, channel_views)
sheet1.write(2, 3, channel_vid_count)

wb.save(f'{channel_name}(channel_stats).xls')